In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator and then re-execute this cell.')
else:
    print(gpu_info)

Wed Jun 23 22:54:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory

ram = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM.'.format(ram))
if ram < 20:
    print('To enable a high-RAM runtime, select the Runtime > "Change runtime type", then select High-RAM in the Runtime shape dropdown '
          'and then re-execute this cell.')
else:
    print('You are using a high-RAM runtime.')

Your runtime has 13.6 gigabytes of available RAM.
To enable a high-RAM runtime, select the Runtime > "Change runtime type", then select High-RAM in the Runtime shape dropdown and then re-execute this cell.


In [4]:
from tensorflow import config

physical_devices = config.list_physical_devices('GPU')
try:
    config.experimental.set_memory_growth(physical_devices[0], True)
except Exception as exception:
    print(exception)

In [5]:
!pip install tensorflow-addons

In [6]:
from tensorflow import device
from tensorflow_addons import layers as new_layers
from tensorflow.keras import losses
from tensorflow.keras import models

import gc
import h5py
import imageio
import numpy as np
import os

In [7]:
model_path = '/content/gdrive/My Drive/Licenta/Traffic4Cast/Istanbul/checkpoints/UNet12/model_39.h5'
with device('gpu:0'):
    model = models.load_model(model_path)
loss = losses.MeanSquaredError()

dynamic_file_path = '/content/gdrive/My Drive/Licenta/Traffic4Cast/Istanbul/files/validation/2019-07-11_istanbul_9ch.h5'
static_file_path = '/content/gdrive/My Drive/Licenta/Traffic4Cast/Istanbul/files/ISTANBUL_static_2019.h5'

In [8]:
def get_data(file_path):
    file = h5py.File(file_path, 'r')
    group_key = list(file.keys())[0]
    data = np.array(file[group_key][:], dtype=np.float32)
    file.close()
    return data

In [9]:
def get_validation_data(dynamic_data, static_data):
    data = np.array(np.split(dynamic_data, 12))
    inputs = data[:, :12, :, :, :]
    outputs = data[:, 12:, :, :, :]
    outputs = np.take(outputs, np.arange(8), axis=-1)
    inputs = np.moveaxis(inputs, 1, -1).reshape((12, 495, 436, -1))
    outputs = np.moveaxis(outputs, 1, -1).reshape((12, 495, 436, -1))
    static_data = np.repeat(np.expand_dims(static_data, axis=0), repeats=12, axis=0)
    inputs = np.concatenate((inputs, static_data), axis=-1)
    return inputs / 255.0, outputs / 255.0

In [10]:
def get_restored_validation_data(data):
    data *= 255.0
    data = np.moveaxis(data.reshape((12, 495, 436, 8, 12)), -1, 1)  # (12, 12, 495, 436, 8)
    return data.reshape((-1, 495, 436, 8))  # (144, 495, 436, 8)

In [11]:
def get_all_day_mask(data):
    data = data.reshape(-1, 495, 436, 9)
    data = np.take(data, [1, 3, 5, 7], axis=-1)
    data = np.moveaxis(data, -1, 1).reshape(-1, 495, 436)
    data = np.mean(data, axis=0, dtype=np.float32)
    return (data > 0.0).astype(np.uint8)

In [12]:
def get_first_half_day_mask(data):
    data = np.array(np.split(data, 12))
    data = data[:, :12, :, :, :]
    data = data.reshape(-1, 495, 436, 9)
    data = np.take(data, [1, 3, 5, 7], axis=-1)
    data = np.moveaxis(data, -1, 1).reshape(-1, 495, 436)
    data = np.mean(data, axis=0, dtype=np.float32)
    return (data > 0.0).astype(np.uint8)

In [13]:
dynamic_data = get_data(dynamic_file_path)
static_data = get_data(static_file_path)
inputs, actual = get_validation_data(dynamic_data, static_data)
with device('gpu:0'):
    prediction = model.predict(inputs)
    prediction = np.clip(prediction, 0.0, 1.0)
gc.collect()
print('loss before reshape:', loss(actual, prediction).numpy())

loss before reshape: 0.00094104046


In [14]:
actual = get_restored_validation_data(actual)
prediction = get_restored_validation_data(prediction)
print('loss after reshape without mask:', loss(actual, prediction).numpy())

loss after reshape without mask: 61.19116


In [15]:
prediction_copy = np.copy(prediction)
mask = get_all_day_mask(dynamic_data)

path = '/content/gdrive/My Drive/Licenta/Traffic4Cast/Istanbul/movies/2019-07-11_istanbul_all_day_mask.png'
imageio.imwrite(path, mask * 255)
print('created image:', path)

for batch in range(144):
    for channel in range(8):
        prediction_copy[batch, :, :, channel] *= mask
result = loss(actual, prediction_copy).numpy()
print('loss after reshape with all day mask:', result)
print('loss before reshape with all day mask:', result / (255.0 ** 2))

created image: /content/gdrive/My Drive/Licenta/Traffic4Cast/Istanbul/movies/2019-07-11_istanbul_all_day_mask.png
loss after reshape with all day mask: 61.155003
loss before reshape with all day mask: 0.000940484468958003


In [16]:
prediction_copy = np.copy(prediction)
mask = get_first_half_day_mask(dynamic_data)

path = '/content/gdrive/My Drive/Licenta/Traffic4Cast/Istanbul/movies/2019-07-11_istanbul_first_half_day_mask.png'
imageio.imwrite(path, mask * 255)
print('created image:', path)

for batch in range(144):
    for channel in range(8):
        prediction_copy[batch, :, :, channel] *= mask
result = loss(actual, prediction_copy).numpy()
print('loss after reshape with first half day mask:', result)
print('loss before reshape with first half day mask:', result / (255.0 ** 2))

created image: /content/gdrive/My Drive/Licenta/Traffic4Cast/Istanbul/movies/2019-07-11_istanbul_first_half_day_mask.png
loss after reshape with first half day mask: 61.15138
loss before reshape with first half day mask: 0.0009404287371255947
